This script calculates auPR, auROC and F1 scores for test portion of the Topic CREs, the non-augmented version to benchmark models against each other

In [ ]:
import os
import sys
import random
import numpy as np
import pandas as pd
import crested
import pickle

In [ ]:
dataDir='/home/DeepHB/'
out_Dir='/home/DeepHB/Outs/'
os.chdir(out_Dir)



In [ ]:
bed_files=dataDir+'TopicMP_CREs'
chromsizes=dataDir+'extrafiles/chromSizes_canchrom.genome'
adata = crested.import_beds(
    beds_folder=bed_files, chromsizes_file=chromsizes
)  

In [ ]:
##Load the test split
with open('Outs/deephb_testsplit_cnn_57MP.pkl', 'rb') as f:
    test_part=pickle.load( f)


In [ ]:
arr1 = test_part.index.astype(str).to_list()
arr2 = df.index.astype(str).to_list()

In [ ]:
common = set(arr1) & set(arr2)
print(len(arr1))
print(len(arr2))
print(len(common))

In [ ]:
# common is test part of adata, it is 10%
adata_subset = adata[:, list(common)]
adata_subset

#### Now loading the models and running prediction to obtain metric values

In [ ]:

import keras
genome_fasta=dataDir+'extrafiles/gencdH38p13r37.fa'



In [ ]:
##load model
model_path = "deephb_cnn_57MP_99.keras"
#model_path = deephb_lstmv1_57MP_53.keras"
#model_path = "deephb_lstmv2_57MP_73.keras"


In [ ]:

model = keras.models.load_model(
    model_path, compile=False
)  #


In [ ]:
prediction = crested.tl.predict(adata_subset, model,genome=genome_fasta)

In [ ]:
with open(os.path.join('HBCRE_test/predHBCRE_test_deephb_cnn_57MP_99.pkl'), 'wb') as f:
      pickle.dump(prediction, f)

In [ ]:
#with open(os.path.join('HBCRE_test/predHBCRE_test_deephb_lstmv1_57MP_53.pkl'), 'wb') as f:
#     pickle.dump(prediction, f)

In [ ]:
#with open(os.path.join('HBCRE_test/predHBCRE_test_deephb_lstmv2_57MP_73.pkl'), 'wb') as f:
#      pickle.dump(prediction, f)

#### Obtianing metric values

In [ ]:
with open(os.path.join(dataDir+'TopicMPs.txt'), "r") as file:
    labels = file.read().splitlines()
print(labels)

In [ ]:
gt=adata_subset.X.T #ground turth
print(gt.shape)
gt_rs=gt.sum(axis=1)


## F1 scores

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt


In [ ]:

# Actual binary labels and predicted scores
# gt: ground truth, shape (n_CREs, 57)
# prediction: obtained score, shape (n_CREs, 57)

best_thresholds = []
best_f1s = []
all_metrics = []

thresholds = np.arange(0.00, 1.01, 0.01)

for i in range(57):  # For each Topic MP 
    y_t = gt[:, i]
    y_s = prediction[:, i]

    best_f1 = 0
    best_thresh = 0
    metrics_for_factor = []

    for thresh in thresholds:
        y_pred = (y_s >= thresh).astype(int)
        precision = precision_score(y_t, y_pred, zero_division=0)
        recall = recall_score(y_t, y_pred, zero_division=0)
        f1 = f1_score(y_t, y_pred, zero_division=0)

        metrics_for_factor.append((thresh, precision, recall, f1))

        if f1 > best_f1:
            best_f1 = f1
            best_thresh = thresh

    best_thresholds.append(best_thresh)
    best_f1s.append(best_f1)
    all_metrics.append(metrics_for_factor)



In [ ]:
### Save best thresholdsimport pandas as pd

summary_data = []
## 67 classes
for idx in range(57):
    # Extract metrics at the best threshold
    best_thresh = best_thresholds[idx]
    metrics = next((m for m in all_metrics[idx] if m[0] == best_thresh), None)

    summary_data.append({
        "MP": labels[idx],
        "Best Threshold": best_thresh,
        "Precision": metrics[1],
        "Recall": metrics[2],
        "F1 Score": metrics[3]
    })

#print(summary_df)


In [ ]:
summary_df = pd.DataFrame(summary_data)
summary_df.to_csv("HBCRE_test/MP_threshold_summary_deephb_cnn_57MP_99.txt", sep='\t')
#summary_df.to_csv("HBCRE_test/MP_threshold_summary_deephb_lstmv1_57MP_53.txt", sep='\t')
#summary_df.to_csv("HBCRE_test/MP_threshold_summary_deephb_lstmv2_57MP_73.txt", sep='\t')


## auPR and auROC values

In [ ]:
adata_subset.layers["predict"] = prediction.T  # adata expects (C, N) instead of (N, C)


In [ ]:
import tensorflow as tf

In [ ]:
auPR_values = []
auROC_values = []


for i in range(gt.shape[1]):
    roc = tf.keras.metrics.AUC(curve='ROC')
    roc.update_state(gt[:, i], prediction[:, i])
    auROC = roc.result().numpy()
    auROC_values.append(auROC)
    
    pr = tf.keras.metrics.AUC(curve='PR')
    pr.update_state(gt[:, i], prediction[:, i])
    auPR = pr.result().numpy()
    auPR_values.append(auPR)
    

    #print(f"Class {labels[i]}: auROC={auROC:.4f}, auPR={auPR:.4f}")

macro_auPR = np.mean(auPR_values)
macro_auROC = np.mean(auROC_values)

print(f"Macro-Average auPR: {macro_auPR}")
print(f"Macro-Average auROC: {macro_auROC}")


In [ ]:
#obtain values for box plot
data = {'ROC': auROC_values, 'PR': auPR_values}
df = pd.DataFrame(data)

# Print the DataFrame (optional)
#print(df)

# Save the DataFrame to a CSV file
df.to_csv('HBCRE_test/ROC_PR_deephb_cnn_99.txt', index=False,sep='\t')
#df.to_csv('HBCRE_test/ROC_PR_deephb_lstmv1_53.txt', index=False,sep='\t')
#df.to_csv('HBCRE_test/ROC_PR_deephb_lstmv2_73.txt', index=False,sep='\t')

